In [9]:
from typing_extensions import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import Field, BaseModel


BASE_URL='http://localhost:11434'
MODEL_NAME= 'llama3.2'

llm = ChatOllama(model=MODEL_NAME, base_url=BASE_URL)

In [10]:
class SentimentAnalysis(BaseModel):
    sentiment : Literal["positive","negative"] = Field(description="The sentiment classification. Either positive or negative.")
    confidence : float = Field(ge=0, le=1.0, description="Confidence score between 0.0 and 1.0.")
    reason: str = Field(description="Brief explanation")


In [11]:
## State Definition

class SentimentState(TypedDict):
    original_tweet: str
    sentiment: str
    confidence: float

    response_tweet: str

In [12]:
## Nodes

def analyze_sentiment(state: SentimentState):
    tweet = state['original_tweet']
    print(f"Analysing customer tweet : {tweet}")

    structured_llm = llm.with_structured_output(SentimentAnalysis)
    messages = [
        SystemMessage("Analyze sentiment and provide the structured output. Use 0 to 1.0 scale for confidence. Lower is negative and higher is positive."),
        HumanMessage(tweet)
    ]
    analysis = structured_llm.invoke(messages)

    print(f"Sentiment Analysis is done:\n{analysis}")
    return {'sentiment': analysis.sentiment, 'confidence': analysis.confidence}


In [13]:
state = {'original_tweet': "Just launched my new product!"}
analyze_sentiment(state)

Analysing customer tweet : Just launched my new product!
Sentiment Analysis is done:
sentiment='positive' confidence=0.8 reason='Excitement and happiness about launching a new product'


{'sentiment': 'positive', 'confidence': 0.8}

In [14]:
def generate_positive_response(state: SentimentState):
    print(f"Current state in positive response node: {state}")

    messages =[SystemMessage(f""""Generate a warm response to this positive tweet under 280 characters.
                             Confidence: {state['confidence']} .High confidence proposes be enthusiastic otherwise be friendly""")
               ,HumanMessage(state['original_tweet'])]
    response = llm.invoke(messages)

    return {'response_tweet': response.content.strip()}
    

In [15]:
def generate_negative_response(state: SentimentState):
    print(f"Current state in negative response node: {state}")

    messages =[SystemMessage(f""""Generate an empathetic response to this negative tweet under 280 characters.
                             If confidence {state['confidence']} is very low then be empathetic otherwise be understanding.""")
               ,HumanMessage(state['original_tweet'])]
    response = llm.invoke(messages)

    return {'response_tweet': response.content.strip()}


In [16]:
def route_by_sentiment(state: SentimentState):
    if state['sentiment'] == 'positive':
        return 'positive_response'
    else:
        return 'negative_response'

In [22]:
## Graph

def create_router_graph():
    builder = StateGraph(SentimentState)

    #add nodes
    builder.add_node("analyze", analyze_sentiment)
    builder.add_node("positive_response", generate_positive_response)
    builder.add_node("negative_response", generate_negative_response)

    # add edges
    builder.add_edge(START, "analyze")

    builder.add_conditional_edges("analyze", route_by_sentiment,['positive_response','negative_response'])
    builder.add_edge("positive_response", END)
    builder.add_edge("negative_response", END)

    graph = builder.compile()

    return graph

In [23]:
graph = create_router_graph()

In [26]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	analyze(analyze)
	positive_response(positive_response)
	negative_response(negative_response)
	__end__([<p>__end__</p>]):::last
	__start__ --> analyze;
	negative_response --> __end__;
	positive_response --> __end__;
	analyze -.-> positive_response;
	analyze -.-> negative_response;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [27]:
tweet = "Just launched my new product! the response from everyone has been amazing so far."
result = graph.invoke({'original_tweet': tweet})

Analysing customer tweet : Just launched my new product! the response from everyone has been amazing so far.
Sentiment Analysis is done:
sentiment='positive' confidence=0.9 reason='None provided'
Current state in positive response node: {'original_tweet': 'Just launched my new product! the response from everyone has been amazing so far.', 'sentiment': 'positive', 'confidence': 0.9}


In [28]:
result

{'original_tweet': 'Just launched my new product! the response from everyone has been amazing so far.',
 'sentiment': 'positive',
 'confidence': 0.9,
 'response_tweet': '"That\'s incredible to hear! Your hard work is really paying off, and it\'s awesome that your team is responding with such enthusiasm! Can\'t wait to see what the future holds for your new product!" #NewProductLaunch #SuccessStory'}

In [29]:
tweet = "Really disappointed with the service I received today."
result = graph.invoke({'original_tweet': tweet})
result

Analysing customer tweet : Really disappointed with the service I received today.
Sentiment Analysis is done:
sentiment='negative' confidence=0.0 reason='disappointed with the service'
Current state in negative response node: {'original_tweet': 'Really disappointed with the service I received today.', 'sentiment': 'negative', 'confidence': 0.0}


{'original_tweet': 'Really disappointed with the service I received today.',
 'sentiment': 'negative',
 'confidence': 0.0,
 'response_tweet': '"I\'m so sorry to hear that your experience was disappointing. That can be really frustrating and upsetting, especially when you\'re not getting the support you need. Would you like me to help you explore what happened or find a solution?"'}